In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,BaggingClassifier,AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score,classification_report,confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import StratifiedKFold,KFold,train_test_split

In [16]:
train=pd.read_csv('ins3_train.csv')
test=pd.read_csv('ins3_test.csv')

In [17]:
train.head()

,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Garden,Settlement,Building Dimension,Building_Type,...,3,4,5,6,7,8,9,>=10,paint_fence,month
0,H14663,2013,1.0,0,1,0,1.0,1,290.0,1,...,0,0,0,0,0,0,0,0,1,12
1,H2037,2015,1.0,0,0,1,0.0,0,490.0,1,...,0,1,0,0,0,0,0,0,0,12
2,H3802,2014,1.0,0,1,0,1.0,1,595.0,1,...,0,0,0,0,0,0,0,0,1,12
3,H3834,2013,1.0,0,0,0,1.0,1,2840.0,1,...,0,0,0,0,0,0,0,0,0,12
4,H5053,2014,1.0,0,0,1,0.0,0,680.0,1,...,1,0,0,0,0,0,0,0,0,12


In [18]:
train.columns

Index(['Customer Id', 'YearOfObservation', 'Insured_Period', 'Residential',
       'Building_Painted', 'Building_Fenced', 'Garden', 'Settlement',
       'Building Dimension', 'Building_Type', 'Date_of_Occupancy', 'Geo_Code',
       'Claim', '   .', '1', '2', '3', '4', '5', '6', '7', '8', '9', '>=10',
       'paint_fence', 'month'],
      dtype='object')

In [19]:
test_id=test['Customer Id']
test_id

0       H11920
1       H11921
2        H9805
3        H7493
4        H7494
         ...  
3064    H11583
3065    H11720
3066    H11721
3067    H12408
3068     H9021
Name: Customer Id, Length: 3069, dtype: object

In [20]:
train=train.drop('Customer Id',axis=1)
test=test.drop('Customer Id',axis=1)

In [21]:
X=train.drop('Claim',axis=1)
y=train.Claim

In [22]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=42)

In [23]:
from sklearn.metrics import auc

In [24]:
model=CatBoostClassifier(n_estimators=800,eval_metric='AUC',max_depth=5,learning_rate=0.1,od_wait=50, 
                              subsample=0.9,bootstrap_type='Bernoulli',metric_period=20,
                     #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     use_best_model=True)

In [25]:
model.fit(X_train, y_train, eval_set=(X_test, y_test))

0:	test: 0.6698251	best: 0.6698251 (0)	total: 208ms	remaining: 2m 46s
20:	test: 0.7115447	best: 0.7115447 (20)	total: 625ms	remaining: 23.2s
40:	test: 0.7139615	best: 0.7140037 (35)	total: 858ms	remaining: 15.9s
60:	test: 0.7144937	best: 0.7144937 (60)	total: 1.08s	remaining: 13.1s
80:	test: 0.7176865	best: 0.7189336 (70)	total: 1.28s	remaining: 11.3s
100:	test: 0.7178998	best: 0.7189336 (70)	total: 1.51s	remaining: 10.4s
120:	test: 0.7163432	best: 0.7189336 (70)	total: 1.89s	remaining: 10.6s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7189335796
bestIteration = 70

Shrink model to first 71 iterations.


In [26]:
pred = model.predict_proba(test)[:, 1]

In [27]:
# Split the data into 30% validation and 70% training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42) #0.15 bank40

In [28]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb=[]
y_pred_totcb=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit, GroupKFold
fold=StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=CatBoostClassifier(n_estimators=800,eval_metric='AUC',max_depth=4,learning_rate=0.1,#reg_lambda=5,#5
                              subsample=0.9,bootstrap_type='Bernoulli',#leaf_estimation_iterations=10,
                    #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     use_best_model=True)
    #CatBoostClassifier(n_estimators=1000,eval_metric='AUC',max_depth=5,learning_rate=0.1,reg_lambda=5,#5
                              #subsample=0.9,bootstrap_type='Bernoulli',
                    #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     #use_best_model=True)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(test)[:, 1]
    y_pred_totcb.append(p)

0:	test: 0.6931422	test1: 0.6746810	best: 0.6746810 (0)	total: 14.6ms	remaining: 11.7s
100:	test: 0.7747940	test1: 0.7024475	best: 0.7041135 (81)	total: 1.63s	remaining: 11.3s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7041134985
bestIteration = 81

Shrink model to first 82 iterations.
err:  0.7041134984908706
0:	test: 0.6473000	test1: 0.6528848	best: 0.6528848 (0)	total: 15.9ms	remaining: 12.7s
100:	test: 0.7669939	test1: 0.7298798	best: 0.7319945 (32)	total: 1.48s	remaining: 10.2s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7319945412
bestIteration = 32

Shrink model to first 33 iterations.
err:  0.7319945412345974
0:	test: 0.6724916	test1: 0.6830082	best: 0.6830082 (0)	total: 15.1ms	remaining: 12.1s
100:	test: 0.7641938	test1: 0.7239983	best: 0.7255865 (72)	total: 1.3s	remaining: 8.98s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7255864989
bestIteration = 72

Shrink model to first 73 iterations.
err:  0.7255

In [29]:
np.mean(errcb)

0.7205648461919408

In [30]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb1=[]
y_pred_totcb1=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
fold=StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m1=CatBoostClassifier(n_estimators=1000,eval_metric='AUC',max_depth=4,learning_rate=0.1,od_wait=50, reg_lambda=3,
                              bootstrap_type='Bayesian',
                     use_best_model=True)
    m1.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m1.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb1.append(roc_auc_score(y_test,preds))
    p1 = m1.predict_proba(test)[:, 1]
    y_pred_totcb1.append(p1)

0:	test: 0.6600821	test1: 0.6385077	best: 0.6385077 (0)	total: 16.4ms	remaining: 16.4s
100:	test: 0.7711234	test1: 0.6974987	best: 0.6982837 (22)	total: 1.2s	remaining: 10.7s
200:	test: 0.8017679	test1: 0.6937025	best: 0.6997729 (129)	total: 2.27s	remaining: 9.01s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6997728835
bestIteration = 129

Shrink model to first 130 iterations.
err:  0.6997728834832502
0:	test: 0.6449946	test1: 0.6585204	best: 0.6585204 (0)	total: 10.1ms	remaining: 10.1s
100:	test: 0.7590990	test1: 0.7314332	best: 0.7349037 (38)	total: 1.41s	remaining: 12.6s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7349037245
bestIteration = 38

Shrink model to first 39 iterations.
err:  0.7349037245116498
0:	test: 0.6556830	test1: 0.6476476	best: 0.6476476 (0)	total: 14.3ms	remaining: 14.3s
100:	test: 0.7641468	test1: 0.7311067	best: 0.7320198 (77)	total: 1.26s	remaining: 11.2s
Stopped by overfitting detector  (100 iterations wait)

b

In [31]:
np.mean(errcb1)

0.7222321437988577

In [32]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
fold=KFold(n_splits=5)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m2=XGBClassifier(max_depth=6, learning_rate=0.1, n_estimators=800, silent=True, metrics='auc',colsample_bylevel=0.8, reg_alpha=0.8)
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m2.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb2.append(roc_auc_score(y_test,preds))
    p2 = m2.predict_proba(test)[:, 1]
    y_pred_totcb2.append(p2)

[09:33:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "metrics", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:33:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.65507	validation_1-logloss:0.65947


C:\Users\bryan\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[100]	validation_0-logloss:0.37910	validation_1-logloss:0.48562
[137]	validation_0-logloss:0.35355	validation_1-logloss:0.49071
err:  0.7319027177842706
[09:33:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "metrics", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:33:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.65575	validation_1-logloss:0.65510


C:\Users\bryan\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
C:\Users\bryan\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
C:\Users\bryan\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[100]	validation_0-logloss:0.37440	validation_1-logloss:0.47354
[128]	validation_0-logloss:0.35486	validation_1-logloss:0.47706
err:  0.7256241015211045
[09:33:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "metrics", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:33:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.65639	validation_1-logloss:0.65410


...

C:\Users\bryan\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
C:\Users\bryan\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[100]	validation_0-logloss:0.37276	validation_1-logloss:0.53941
[122]	validation_0-logloss:0.36074	validation_1-logloss:0.54390
err:  0.6949319630598234
[09:33:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "metrics", "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:33:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.65421	validation_1-logloss:0.65737


C:\Users\bryan\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
C:\Users\bryan\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[100]	validation_0-logloss:0.37131	validation_1-logloss:0.52309
[120]	validation_0-logloss:0.35673	validation_1-logloss:0.52462
err:  0.6824726971677091


C:\Users\bryan\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [33]:
np.mean(errcb2)

0.7096437241327703

In [34]:
d = {"Customer Id": test_id, 'Claim': np.mean(y_pred_totcb, 0)}
test_predictions = pd.DataFrame(data=d)
test_predictions = test_predictions[["Customer Id", 'Claim']]

In [35]:
test_predictions.to_csv('in14.csv', index=False)

In [37]:
test_predictions.head()

,Customer Id,Claim
0,H11920,0.096764
1,H11921,0.085731
2,H9805,0.086794
3,H7493,0.148120
4,H7494,0.129505


In [38]:
d = {"Customer Id": test_id, 'Claim': np.mean(y_pred_totcb1, 0)}
test_predictioned = pd.DataFrame(data=d)
test_predictioned = test_predictioned[["Customer Id", 'Claim']]

In [39]:
test_predictioned.head()

,Customer Id,Claim
0,H11920,0.096945
1,H11921,0.085889
2,H9805,0.076101
3,H7493,0.142849
4,H7494,0.126811


In [40]:
d = {"Customer Id": test_id, 'Claim': np.mean(y_pred_totcb2, 0)}
test_prediction = pd.DataFrame(data=d)
test_prediction = test_prediction[["Customer Id", 'Claim']]

In [41]:
test_predictions.head()

,Customer Id,Claim
0,H11920,0.096764
1,H11921,0.085731
2,H9805,0.086794
3,H7493,0.148120
4,H7494,0.129505


In [46]:
a = pd.read_csv('in14.csv')
b = pd.read_csv('i2.csv')

In [47]:
b['Claim'].corr(a['Claim'])

0.9810561365970896

In [48]:
test_predictions['Claim'].corr(test_prediction['Claim'])

0.9708649052297122

In [49]:
test_predictions['Claim'] = (a['Claim'] * 0.69 + b['Claim'] * 0.31).round(2)

In [50]:
test_predictions.to_csv('ins_stack8.csv', index=False)